# Dialogue and Narrative Coursework - Subtask 1 - New train Dataset

In [1]:
## imports
# %%capture
# !pip install datasets

from datasets import load_dataset, Dataset
import pandas as pd

In [2]:
## A function to load a specific doc2dial dataset

def load_doc2dial_dataset(name='dialogue_domain', split='train'):
  cache_dir = "./data_cache"

  return load_dataset(
      "doc2dial",
      name=name,
      split=split,
      ignore_verifications=True,
      cache_dir=cache_dir,
  )


In [3]:
train_dialogues = load_doc2dial_dataset(name="dialogue_domain", split="train")
val_dialogues = load_doc2dial_dataset(name="dialogue_domain", split="validation")

documents = load_doc2dial_dataset(name="document_domain", split="train")

train_data = load_doc2dial_dataset(name="doc2dial_rc", split="train")
val_data = load_doc2dial_dataset(name="doc2dial_rc", split="validation")

Reusing dataset doc2dial (./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee)
Reusing dataset doc2dial (./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee)
Reusing dataset doc2dial (./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee)
Reusing dataset doc2dial (./data_cache/doc2dial/doc2dial_rc/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee)
Reusing dataset doc2dial (./data_cache/doc2dial/doc2dial_rc/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee)


In [4]:
train_data[0]

{'answers': {'answer_start': [346],
  'text': ['you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. ']},
 'context': 'Many DMV customers make easily avoidable mistakes that cause them significant problems, including encounters with law enforcement and impounded vehicles. Because we see customers make these mistakes over and over again , we are issuing this list of the top five DMV mistakes and how to avoid them. \n\n1. Forgetting to Update Address \nBy statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the poli

In [5]:
def add_spans(sample, dialogues):
  dial_id, turn_id = sample['id'].split('_')

  dial = dialogues.filter(lambda ex: ex['dial_id'] == dial_id)[0]

  doc_id = dial['doc_id']   ## corresponding document id
  span_ids = []             ## corresponding span ids

  for tr in dial['turns']:
    if tr['turn_id'] == int(turn_id)+1:

      for sp in tr['references']:
        span_ids.append(sp['sp_id'])

      break

  doc = documents.filter(lambda ex: ex['doc_id'] == doc_id)[0]
  
  spans = {}
  answer_spans = {}
  for span in doc['spans']:
    spans[span['id_sp']] = span['text_sp']
    if span['id_sp'] in span_ids:
      answer_spans[span['id_sp']] = span['text_sp']

  sample['spans'] = spans
  sample['answers']['spans'] = answer_spans

  return sample

In [7]:
new_data_train = train_data.map(add_spans)

new_data_train[0]

TypeError: add_spans() missing 1 required positional argument: 'dialogues'

In [6]:
from tqdm import tqdm

new_train_data = pd.DataFrame(columns=['id', 'question', 'context', 'answers', 'spans',  'domain', 'title'])

for ex in tqdm(train_data):
  new_train_data = new_train_data.append(add_spans(ex, train_dialogues), ignore_index=True)

new_train_data.head()

  0%|                                                           | 0/20598 [00:00<?, ?it/s]Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-63f6f8de6047c1eb.arrow


2022-01-07 15:18:18.741728: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-07 15:18:18.741749: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
  0%|                                                | 1/20598 [00:05<33:47:31,  5.91s/it]Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-63f6f8de6047c1eb.arrow
Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-3bb25d0ca9aa4180.arrow
Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-63f6f8de6047c1eb.arrow


Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-3bb25d0ca9aa4180.arrow
  0%|                                                | 6/20598 [00:07<24:01:34,  4.20s/it]Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-235346d6b22eb777.arrow
Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-3bb25d0ca9aa4180.arrow
Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-235346d6b22eb777.arrow
Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-3bb25d0ca9aa4180.arrow
Loading cached processed dataset at ./da

Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-3bb25d0ca9aa4180.arrow
  0%|                                               | 11/20598 [00:08<17:09:31,  3.00s/it]Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-a9e10c5fdf770345.arrow
Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-3bb25d0ca9aa4180.arrow
Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-a9e10c5fdf770345.arrow
Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-3bb25d0ca9aa4180.arrow
Loading cached processed dataset at ./da

Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-3bb25d0ca9aa4180.arrow
  0%|                                               | 16/20598 [00:09<12:20:59,  2.16s/it]Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-e9f91a552821294c.arrow
Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-3bb25d0ca9aa4180.arrow
Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-e9f91a552821294c.arrow
Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-3bb25d0ca9aa4180.arrow
Loading cached processed dataset at ./da

...

Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-eddfa2a81d607ddf.arrow
 12%|█████▍                                        | 2456/20598 [11:27<1:04:59,  4.65it/s]Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-4b665264c362dd96.arrow
Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-eddfa2a81d607ddf.arrow
Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-4b665264c362dd96.arrow
Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-eddfa2a81d607ddf.arrow
Loading cached processed dataset at ./da

Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-eddfa2a81d607ddf.arrow
 12%|█████▋                                          | 2463/20598 [11:28<59:07,  5.11it/s]Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-e0dff658c93add01.arrow
Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-eddfa2a81d607ddf.arrow
Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-e0dff658c93add01.arrow
Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-eddfa2a81d607ddf.arrow
Loading cached processed dataset at ./da

Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-eddfa2a81d607ddf.arrow
 12%|█████▊                                          | 2469/20598 [11:29<55:19,  5.46it/s]Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-4d2ebf2e6ad89d87.arrow
Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-eddfa2a81d607ddf.arrow
Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-4d2ebf2e6ad89d87.arrow
Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee/cache-eddfa2a81d607ddf.arrow
Loading cached processed dataset at ./da

 12%|█████▌                                        | 2474/20598 [11:33<1:24:40,  3.57it/s]

KeyboardInterrupt: 

In [ ]:
new_train_data.to_csv('doc2dial_rc_train.csv')
# new_train_data = pd.read_csv('doc2dial_rc_train.csv')
new_train_data.head()

In [ ]:
new_train_data

In [ ]:
# from tqdm import tqdm

# new_val_data = pd.DataFrame(columns=['id', 'question', 'context', 'answers', 'spans',  'domain', 'title'])

# for ex in tqdm(val_data):
#   new_val_data = new_val_data.append(add_spans(ex, val_dialogues), ignore_index=True)

# new_val_data.head()

In [ ]:
# new_val_data.to_csv('doc2dial_rc_val.csv')